In [1]:
%pip install -q -r requirements.txt

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
# Imports and Environment Setup
import os
from langchain_huggingface import HuggingFaceEndpoint, HuggingFaceEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import FAISS
import pypdf

from dotenv import load_dotenv, find_dotenv
from pyfiglet import figlet_format
from termcolor import colored

load_dotenv(find_dotenv())
#print("Token:", os.environ.get("HF_TOKEN"))

HF_TOKEN = os.environ.get("HF_TOKEN")
HUGGINGFACE_REPO_ID = "mistralai/Mistral-7B-Instruct-v0.3"
DB_FAISS_PATH = "vectorstore/db_faiss"

In [3]:
# Stylish Heading
def stylish_heading():
    title = figlet_format("AI-ChatBot", font="starwars", width=1000)
    print(colored(title, "green"))

#stylish_heading()

In [4]:
HUGGINGFACE_REPO_ID="mistralai/Mistral-7B-Instruct-v0.3"

def load_llm(huggingface_repo_id):
    llm = HuggingFaceEndpoint(
        repo_id=huggingface_repo_id,
        temperature=0.5,
        huggingfacehub_api_token=HF_TOKEN,
    )
    return llm

In [5]:
CUSTOM_PROMPT_TEMPLATE = """
Use the pieces of information provided in the context to answer user's question.
If you dont know the answer, just say that you dont know, dont try to make up an answer. 
Dont provide anything out of the given context

Context: {context}
Question: {question}

Start the answer directly. No small talk please.
"""

def set_custom_prompt(custom_prompt_template):
    prompt=PromptTemplate(template=custom_prompt_template, input_variables=["context", "question"])
    return prompt


In [6]:
DATA_PATH="data/"
def load_pdf_files(data):
    loader = DirectoryLoader(data,
                             glob='*.pdf',
                             loader_cls=PyPDFLoader)
    
    documents=loader.load()
    return documents

documents=load_pdf_files(data=DATA_PATH)
#print("Length of PDF pages: ", len(documents))

In [7]:
def create_chunks(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,
                                                 chunk_overlap=50)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

text_chunks=create_chunks(extracted_data=documents)
#print("Length of Text Chunks: ", len(text_chunks))

In [8]:
def get_embedding_model():
    embedding_model=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embedding_model

embedding_model=get_embedding_model()

In [9]:
DB_FAISS_PATH="vectorstore/db_faiss"
db=FAISS.from_documents(text_chunks, embedding_model)
db.save_local(DB_FAISS_PATH)

In [10]:
DB_FAISS_PATH="vectorstore/db_faiss"
embedding_model=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
db=FAISS.load_local(DB_FAISS_PATH, embedding_model, allow_dangerous_deserialization=True)

In [13]:
qa_chain=RetrievalQA.from_chain_type(
    llm=load_llm(HUGGINGFACE_REPO_ID),
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={'k':3}),   # Find Top Number of Searches
    return_source_documents=True,
    chain_type_kwargs={'prompt':set_custom_prompt(CUSTOM_PROMPT_TEMPLATE)}
)

In [16]:
# Interactive Chat Loop with Heading, User Echo, and Response

stylish_heading()  # Show the AI-ChatBot banner before chat starts

print("\n🟢 You can start chatting now. Type 'Exit the Chatbot' to end the session.\n")

while True:
    user_query = input("🧠 You: ")
    user = user_query  # Store user input

    if user.strip().lower() == "exit the chatbot":
        print("\n👋 Exiting... Have a great day!\n")
        break

    print(f"\n👤 User: {user}")  # Print what user just said

    try:
        response = qa_chain.invoke({'query': user})
        print(f"🤖 Bot: {response['result']}\n")
    except Exception as e:
        print("❌ Error while processing:", e)

     ___       __           ______  __    __       ___   .___________..______     ______   .___________.
    /   \     |  |         /      ||  |  |  |     /   \  |           ||   _  \   /  __  \  |           |
   /  ^  \    |  |  ______|  ,----'|  |__|  |    /  ^  \ `---|  |----`|  |_)  | |  |  |  | `---|  |----`
  /  /_\  \   |  | |______|  |     |   __   |   /  /_\  \    |  |     |   _  <  |  |  |  |     |  |     
 /  _____  \  |  |        |  `----.|  |  |  |  /  _____  \   |  |     |  |_)  | |  `--'  |     |  |     
/__/     \__\ |__|         \______||__|  |__| /__/     \__\  |__|     |______/   \______/      |__|     
                                                                                                        


🟢 You can start chatting now. Type 'Exit the Chatbot' to end the session.


👤 User: How to cure fever?
🤖 Bot: 
The most effective treatment for a fever is to address its underlying cause, such as through the administration of antibiotics if it's caused by bacteri